In [36]:
import os
import logging
import pandas as pd
from slack_sdk import WebClient  # Import WebClient from Python SDK (github.com/slackapi/python-slack-sdk)
import time


## Read Conversations

conversations = pd.read_csv('data/conversations_history.csv')

## Read Users

users = pd.read_csv('data/slack_users.csv')

## Read Channels

channels = pd.read_csv('data/channels.csv')


,username,email,status,billing-active,has-2fa,has-sso,userid,fullname,displayname,expiration-timestamp
0,jasonlally,jason.lally@gmail.com,Owner,0,0,0,U042ZA3GX,Jason Lally,"Jason Lally (State of CA, he/him)",NaN
1,julia,julia@codeforsanfrancisco.org,Member,0,0,0,U04301E01,Julia Gitis,julia,NaN
2,jesse,jesse@codeforsanfrancisco.org,Member,0,0,0,U0431NL8E,Jesse Biroscak,jesse,NaN
3,judy,judyvansoldt@gmail.com,Member,0,1,0,U043262MY,Judy van Soldt,judy,NaN
4,kbnammacher,kbnammacher@codeforsanfrancisco.org,Member,0,0,0,U043F5GFW,Katherine Nammacher,kbnammacher,NaN
...,...,...,...,...,...,...,...,...,...,...
3513,justinng123409,justinng123409@yahoo.com,Member,1,0,0,U024YN4R7L1,Justin Ng,Justin Ng,NaN
3514,johnjordan1985,johnjordan1985@gmail.com,Member,1,0,0,U024YP1DHPB,john-jordan,john-jordan,NaN
3515,anthonychian,anthonychian@gmail.com,Member,1,0,0,U024YUL7JSD,Anthony Chian,Anthony Chian,NaN
3516,cameronsasai,cameronsasai@gmail.com,Member,1,0,0,U025RG6H82C,Cameron Sasai,Cameron Sasai,NaN


In [53]:
# Find Last 50 MActive Users

conversations_max = conversations.groupby(['user'], as_index = False).agg({'ts':'max'})
conversations_max.sort_values(by = 'ts', ascending = False, inplace = True)
conversations_joined = pd.merge(conversations_max, users, how = 'left', left_on = ['user'], right_on = ['userid'])

conversations_joined [['user','username' ,'email','ts']].head(50)



,username,email,status,billing-active,has-2fa,has-sso,userid,fullname,displayname,expiration-timestamp
0,jasonlally,jason.lally@gmail.com,Owner,0,0,0,U042ZA3GX,Jason Lally,"Jason Lally (State of CA, he/him)",NaN


In [ ]:
# Discuss New Member to New Channel